# Functions

## Standard Functions

Functions are used to give a name to repetitive code that needs to be used more than once with different parameters. If you ever find yourself copying and pasting code just to change a few variables, consider writing a function. Functions in Python are set up simply as shown below:

In [3]:
def my_function(arg1, arg2):
    if arg1 > arg2:
        print('arg1 is greater than arg2')
    else:
        print('arg1 is less than arg2')

my_function(3, 4)

arg1 is less than arg2


The `return` keyword can be used to return one or more outputs from a function. If more than one output is specified, the return type will be a `tuple` containing the objects. If no object is specified to be returned or no return statement is used, the function will automatically return `None`.

In [11]:
def find_x(my_list, x):
    # Return True if the value of x is in the list, False if not
    return x in my_list

print(find_x([0, 'a', 5], 'a'))
print(find_x([0, 2, 4], 3))

True
False


In [12]:
def get_max_val_and_index(my_list):
    # Return the maximum value and index of that value in a list
    
    current_max_index = 0
    # enumerate() returns the index and value when iterated over
    # It is what is known as a generator
    # Later in this chapter we will code enumerate() from scratch
    for n, val in enumerate(my_list):
        if val > my_list[current_max_index]:
            current_max_index = n
    return current_max_index, my_list[current_max_index]
    
    # A more efficient solution to this problem is:
    return max(enumerate(my_list), key=lambda x: x[1])
    # However this requires the use of a lambda statement 
    # which we will talk about in the Anonymous Functions section.
    # Note that this second return statement will not run
    # since one before it already ran

print(get_max_val_and_index([0, 9, 20, 2, 4]))

(2, 20)


## Argument Unpacking

## Anonymous Functions

Sometimes, you may need a quick one line function to do something. The `lambda` keyword defines a callable function that isn't required to be given a name, hence, anonymous.

In [8]:
power = lambda b, e: b**e
print(power(2, 3))

8


Here, `power` is defined as a function that takes a base and an exponent and returns the result of exponentiation. However, didn't I say these were supposed to be anonymous? Let's design a more useful result. Let's say I have a list of numbers and I want to return the cube of all of them quickly. We can do this in one line by leveraging the `map()` function and a `lambda`. `map(func, iterable)` takes a function and applies it to each element in the provided `iterable` and returns a generator (which will be covered in the next section, so for now we will coerce it into a list).

In [9]:
cubes = list(map(lambda x: x**3, [1, 2, 3, 4,]))
print(cubes)

[1, 8, 27, 64]


`filter(func, iterable)` is another useful function that is most often paired with lambdas. It can filter an `iterable` based upon the output of `func`. If `func` returns `True`, the value is placed in the output, otherwise it is ignored. Let's say we only want numbers >=3.

In [10]:
min3 = list(filter(lambda x: x >= 3, [1, 2, 3, 4,]))
print(min3)

[3, 4]


## Generators

Generators can be thought of as functions can pause periodically to `yield` a value and then can continue execution from where they left off when called again. Common generators that you have already seen are `enumerate()` and `range()`. Let's break down how these two work.

In [42]:
def my_range(*args):
    # Duplicate the functionality of the built in function: range()
    # range accepts up to 3 arguments
    num_args = len(args)
    if num_args == 3:  # If we have 3, we got start, stop, and step
        start, stop, step = args
    elif num_args == 2:  # With 2, we only got start and stop
        start, stop = args
        step = 1
    elif num_args == 1:  # With just 1, we only got stop
        stop = args[0]
        start = 0
        step = 1
    else:  # In any other case, raise an error.
        if num_args > 3:
            raise TypeError(
                "my_range expected at most 3 arguments, "
                "got %" % num_args
            )
        else:
            raise TypeError(
                "my_range expected 1 argument, got %s" % num_args
            )
    
    # Start our counter at `start`. 
    # We could also just use `start` as a counter and increment that
    # but this is clearer.
    i = start
    while i < stop:
        # Now here is what makes a generator a generator. 
        # yield pauses execution. The next() time the generator
        # is advanced, it will start here.
        yield i
        i += step
        
# First we initialize our generator
up_to_50_by_10s = my_range(0, 50, 10)
# Then, we can run the next iteration with the next() function
# We know it should iterate 5 times, so let's do that:
print(next(up_to_50_by_10s))
print(next(up_to_50_by_10s))
print(next(up_to_50_by_10s))
print(next(up_to_50_by_10s))
print(next(up_to_50_by_10s))

0
10
20
30
40


Now what happens when we call `next()` again on `up_to_50_by_10s`? The `while` loop is going to end and the function will return. So what should `next()` return?

In [43]:
next(up_to_50_by_10s)

StopIteration: 

As you can see, an exception was raised! We can catch this error when iterating manually to make sure we don't crash our program. In fact, that is precisely what a `for` loop does. It repeatedly calls the `next()` function on an iterator until `StopIteration` is raised and then the `for` loop ends. The output of `range()` and `my_range()` should be exactly the same. (Note: `for` technically doesn't call `next()`, but rather it calls the same stuff that `next()` does. However for practical purposes this distinction doesn't matter.)

In [44]:
print("range demo")
for i in range(2, 6):
    print(i)
    
print("my_range demo")
for i in my_range(2, 6):
    print(i)

range demo
2
3
4
5
my_range demo
2
3
4
5


Now, onto `enumerate()`. This function takes an iterable and returns the index and the value of each element. You can also specify a `start` value as of Python 2.6.

In [45]:
def my_enumerate(iterable, start=0):
    # Duplicate the functionality of the built-in function: enumerate()
    # This function is much simpler than range as it just returns
    # the index and value of each element
    i = start
    for val in iterable:
        yield i, val
        i += 1

enum = my_enumerate(['a', 'b', 'c'], start=99)
print(next(enum))
print(next(enum))
print(next(enum))

(99, 'a')
(100, 'b')
(101, 'c')


Here you can see that just like `return`, we are able to send multiple values back. However, what if we also wanted to be able to update a variable inside of a generator once it has been initialized? For this, `yield` actually has a return value of its own. Let's make an extension to `my_enumerate()` that allows you to change `i` at will.

In [46]:
def my_enumerate2(iterable, start=0):
    # Duplicate the functionality of the built-in function: enumerate()
    # This function is much simpler than range as it just returns
    # the index and value of each element
    i = start
    for val in iterable:
        i = yield i, val
        i += 1

enum = my_enumerate2(['a', 'b', 'c'], start=99)
print(next(enum))
print(enum.send(1))
print(enum.send(1053))

(99, 'a')
(2, 'b')
(1054, 'c')


Because of the rules of Python, we must call `next()` to start the iterations. However after that, we can use `enum.send(val)` to send in a value to the generator. The value gets placed in `i` due to the assignment and the next iteration happens. When assignment from `yield` occurs, the the pause point is at the equals sign. `i` will be updated when `.send()` is called again.

## Decorators